In [1]:
import sys
print(sys.executable)

/usr/bin/python3


In [1]:
import numpy as np
# import os
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import pandas as pd
import importlib
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import tensorflow as tf
import keras
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import utils_ray

from utils_ray import scale_columns, create_windows, append_segments

In [2]:
importlib.reload(utils_ray) 
from utils_ray import scale_columns, create_windows, append_segments

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  0
[]


In [23]:
df = pd.read_csv("movementSensorData.csv")
df = df.rename(columns={'Unnamed: 0': 'time_ms'})
df = append_segments(df)

In [24]:
df_copy = df.copy()

# Replace specific activity value
df_copy['activity'].replace({77: 0}, inplace=True)

# Normalize specific columns
columns_to_normalize = ['lw_x', 'lw_y', 'lw_z']
scaler, df_copy = scale_columns(df_copy, columns_to_normalize)

# Creating windows with optimized function
window_size = 5
X, y = create_windows(df_copy, window_size)


In [25]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42) 
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

In [26]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(355464, 5, 3)
(76171, 5, 3)
(76172, 5, 3)


In [21]:
# INSPECT DATA ORIGINAL VERSUS WINDOWS
df[['lw_x',	'lw_y'	,'lw_z', 'activity']].head(6)


,lw_x,lw_y,lw_z,activity
0,-0.188,-0.941,-0.316,2
1,-0.121,-0.879,-0.320,2
2,-0.070,-0.852,-0.305,2
3,-0.023,-0.879,-0.277,2
4,0.008,-0.941,-0.242,2
5,0.039,-1.012,-0.191,2


In [22]:
print(X[0])
print(y[0])
scaler.inverse_transform(X[0])


[[ 0.04446353 -1.26252832  0.36440705]
 [ 0.17230926 -1.1773578   0.35538717]
 [ 0.26962468 -1.14026741  0.38921172]
 [ 0.35930751 -1.1773578   0.4523509 ]
 [ 0.41846001 -1.26252832  0.53127487]]
2


array([[-0.188, -0.941, -0.316],
       [-0.121, -0.879, -0.32 ],
       [-0.07 , -0.852, -0.305],
       [-0.023, -0.879, -0.277],
       [ 0.008, -0.941, -0.242]])

In [5]:
# Splitting the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

# Determine the number of unique classes
num_classes = df['activity'].nunique()

In [14]:
# Building the LSTM model for multi-class classification
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(window_size, 3)))
model.add(LSTM(50))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Configure Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)

# Training the model with validation data and Early Stopping
model.fit(X_train, y_train, epochs=100, validation_data=(
    X_val, y_val), callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Ensure X_test and y_test are defined and preprocessed as they were for training
predictions = model.predict(X_test)

# If your model outputs probabilities (common in classification), you might need to convert these to class labels
# Adjust if your model's output differs
predicted_labels = np.argmax(predictions, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Additionally, print a classification report for more insights
print("\nClassification Report:")
print(classification_report(y_test, predicted_labels))
print(classification_report(y_test, predicted_labels))


Epoch 1/100
11109/11109 [==============================] - 33s 3ms/step - loss: 0.2592 - accuracy: 0.9208 - val_loss: 0.2294 - val_accuracy: 0.9275
Epoch 2/100
11109/11109 [==============================] - 30s 3ms/step - loss: 0.2288 - accuracy: 0.9274 - val_loss: 0.2374 - val_accuracy: 0.9199
Epoch 3/100
11109/11109 [==============================] - 30s 3ms/step - loss: 0.2213 - accuracy: 0.9290 - val_loss: 0.2185 - val_accuracy: 0.9299
Epoch 4/100
11109/11109 [==============================] - 30s 3ms/step - loss: 0.2150 - accuracy: 0.9304 - val_loss: 0.2083 - val_accuracy: 0.9328
Epoch 5/100
11109/11109 [==============================] - 30s 3ms/step - loss: 0.2100 - accuracy: 0.9315 - val_loss: 0.2113 - val_accuracy: 0.9318
Epoch 6/100
11109/11109 [==============================] - 30s 3ms/step - loss: 0.2048 - accuracy: 0.9331 - val_loss: 0.2043 - val_accuracy: 0.9333
Epoch 7/100
11109/11109 [==============================] - 30s 3ms/step - loss: 0.2002 - accuracy: 0.9343 - val_

In [13]:
for window_size in [10, 20]:    
    X, y = create_windows(df_copy, window_size)# Splitting the dataset into training, validation, and testing sets
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42)

    # Determine the number of unique classes
    num_classes = df['activity'].nunique()
        # Building the LSTM model for multi-class classification
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(window_size, 3)))
    model.add(LSTM(50))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Configure Early Stopping
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)

    # Training the model with validation data and Early Stopping
    model.fit(X_train, y_train, epochs=100, validation_data=(
        X_val, y_val), callbacks=[early_stopping])

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print('WINDOW SIZE = ', window_size)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    # Ensure X_test and y_test are defined and preprocessed as they were for training
    predictions = model.predict(X_test)

    # If your model outputs probabilities (common in classification), you might need to convert these to class labels
    # Adjust if your model's output differs
    predicted_labels = np.argmax(predictions, axis=1)

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test, predicted_labels)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Additionally, print a classification report for more insights
    print("\nClassification Report:")
    print(classification_report(y_test, predicted_labels))
    print(classification_report(y_test, predicted_labels))


Epoch 1/100
11108/11108 [==============================] - 47s 4ms/step - loss: 0.2453 - accuracy: 0.9242 - val_loss: 0.2164 - val_accuracy: 0.9312
Epoch 2/100
11108/11108 [==============================] - 45s 4ms/step - loss: 0.2069 - accuracy: 0.9335 - val_loss: 0.1958 - val_accuracy: 0.9366
Epoch 3/100
11108/11108 [==============================] - 44s 4ms/step - loss: 0.1867 - accuracy: 0.9392 - val_loss: 0.1805 - val_accuracy: 0.9414
Epoch 4/100
11108/11108 [==============================] - 44s 4ms/step - loss: 0.1728 - accuracy: 0.9431 - val_loss: 0.1658 - val_accuracy: 0.9451
Epoch 5/100
11108/11108 [==============================] - 47s 4ms/step - loss: 0.1617 - accuracy: 0.9461 - val_loss: 0.1588 - val_accuracy: 0.9473
Epoch 6/100
11108/11108 [==============================] - 48s 4ms/step - loss: 0.1521 - accuracy: 0.9492 - val_loss: 0.1506 - val_accuracy: 0.9497
Epoch 7/100
11108/11108 [==============================] - 44s 4ms/step - loss: 0.1443 - accuracy: 0.9515 - val_

In [12]:
# Splitting the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

# Determine the number of unique classes
num_classes = df['activity'].nunique()

# Load the model from the pickle file
filename = 'files/model.pkl'
with open(filename, 'rb') as file:
    model = pickle.load(file)

# Ensure X_test and y_test are defined and preprocessed as they were for training
predictions = model.predict(X_test)

# If your model outputs probabilities (common in classification), you might need to convert these to class labels
# Adjust if your model's output differs
predicted_labels = np.argmax(predictions, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Additionally, print a classification report for more insights
print("\nClassification Report:")
print(classification_report(y_test, predicted_labels))
print(classification_report(y_test, predicted_labels))

ValueError: Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['lstm/lstm_cell/kernel:0', 'lstm/lstm_cell/recurrent_kernel:0', 'lstm/lstm_cell/bias:0']

In [4]:
import pickle
filename = 'model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)
    
import pickle
filename = 'files/scaler.pkl'
with open(filename, 'wb') as file:
    pickle.dump(scaler, file)

NameError: name 'model' is not defined

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
# Check if a GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
from utils_ray import scale_columns, create_windows, append_segments
 

Using device: cuda


In [28]:
def create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test, batch_size=32):
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
    test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

In [29]:
df = pd.read_csv("movementSensorData.csv")
df = df.rename(columns={'Unnamed: 0': 'time_ms'})
df = append_segments(df)
df_copy = df.copy()
df_copy['activity'].replace({77: 0}, inplace=True)
# Normalize specific columns
columns_to_normalize = ['lw_x', 'lw_y', 'lw_z']
scaler, df_copy = scale_columns(df_copy, columns_to_normalize)

In [39]:
num_classes = 5  # Update this with your actual number of classes
input_size = 3  # Update this with your actual number of features
hidden_size = 50
num_epochs = 100
patience = 3
batch_size = 32
window_size = 100

X, y = create_windows(df_copy, window_size)

# Splitting the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

train_loader, val_loader, test_loader = create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test, batch_size)

In [40]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(355132, 100, 3)
(76100, 100, 3)
(76100, 100, 3)
